# Cooking Assistant

## openai
The library that will make it easier for us to connect directly to the openAI API.

## os and dotenv
The libraries that allow me to upload this directly to github without exposing my API credentials.

In [1]:
import csv
import os
import openai
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

## Connecting to the openAI API

In [3]:
openai.api_key = os.getenv("OPEN_API_KEY")

## generate_recipe
This function formats the prompt that is sent to GPT3 (text-davinci-003). It takes variables that will eventually be available inputs on a webpage.

In [4]:
def generate_recipe(cooking_skill, place_city, preparation_time, time_of_day):
    prompt = f"Please provide a detailed {time_of_day} cooking recipe with ingredients and step-by-step directions, suitable for someone with {cooking_skill}, originating from {place_city}, and taking {preparation_time} time to prepare and cook. Please also keep it under 10 ingredients. \nPlease provide this in JSON format with the following variables:\nname, description, ingredients, directions. Do not return 'difficulty, origin, time'"

    response = openai.Completion.create(
        engine='text-davinci-003',
        prompt=prompt,
        max_tokens=1000,
        n=1,
        temperature=0,
        top_p=0.5,
        frequency_penalty=0,
        presence_penalty=0
    )
    # return response.choices[0].text.strip()
    return response

In [61]:
cooking_skill = input("Choose your cooking skill (Beginner/Intermediate/Advanced): ")
place_city = input("Choose a city or place that you'd like your meal based on: ")
preparation_time = input("Choose the time it will take to cook (Short/Medium/Long): ")
time_of_day = input("Choose the time of day you'd like the meal to be based on (Breakfast/Lunch/Dinner): ")

## recipe_json
Here we will be executing the function by passing our 3 variables into the prompt, which is then sent to openAPI

In [62]:
recipe = generate_recipe(cooking_skill, place_city, preparation_time, time_of_day)

## JSON Structure
This JSON response will be used to fill in our webpage in an digestible format.

In [63]:
print(recipe)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\n{\n    \"name\": \"Salt Lake City Grilled Cheese\",\n    \"description\": \"A classic grilled cheese sandwich with a Salt Lake City twist!\",\n    \"ingredients\": [\n        \"2 slices of sourdough bread\",\n        \"2 slices of cheddar cheese\",\n        \"2 slices of bacon\",\n        \"1 tablespoon of butter\",\n        \"1 teaspoon of garlic powder\"\n    ],\n    \"directions\": [\n        \"1. Preheat a skillet over medium heat.\",\n        \"2. Spread butter on one side of each slice of bread.\",\n        \"3. Place one slice of bread, butter side down, in the skillet.\",\n        \"4. Top with one slice of cheese, bacon, and the other slice of cheese.\",\n        \"5. Sprinkle garlic powder over the top of the cheese.\",\n        \"6. Place the other slice of bread, butter side up, on top of the cheese.\",\n        \"7. Cook for 3-4 minutes, or until the bottom sl

In [64]:
strip_recipe = recipe.choices[0].text.strip()

In [65]:
import json

In [66]:
recipe_json = json.loads(strip_recipe)

In [69]:
from supabase import create_client, Client

url = os.getenv("SUPABASE_URL")
supakey = os.getenv("SUPABASE_API_KEY")
supabase = create_client(url, supakey)

In [70]:
suparesponse = supabase.table('main').select("name").execute()

In [71]:
suparesponse

APIResponse(data=[{'name': 'Waco-Style BBQ Chicken Sandwich'}, {'name': 'Czech Potato Salad'}, {'name': 'Czech Potato Salad'}, {'name': 'Miami-Style Grilled Fish Tacos'}, {'name': 'Quebec-Style French Toast'}, {'name': 'Detroit-Style Coney Dogs'}, {'name': 'New Orleans Jambalaya'}, {'name': 'No-Cook Fruit Salad'}], count=None)

In [72]:
data = supabase.auth.sign_in_with_password({"email": os.getenv("SUPABASE_EMAIL"), "password": os.getenv("SUPABASE_LOGIN")})

In [73]:
table = supabase.table('main')

In [74]:
insert_query = table.insert({
    'name': recipe_json['name'],
    'description': recipe_json['description'],
    'ingredients': ", ".join(recipe_json['ingredients']),
    'directions': ", ".join(recipe_json['directions']),
    'cooking_skill': cooking_skill,
    'place_city': place_city,
    'preparation_time': preparation_time,
    'time_of_day': time_of_day
})

In [75]:
result = insert_query.execute()

In [76]:
recipe.usage['total_tokens']

348

In [105]:
test = supabase.table('main').select('*').order('id',desc=True).limit(1).execute()
test = test.data[0]
print(type(test))
test

<class 'dict'>


{'id': 20,
 'created_at': '2023-04-13T03:34:50.676323+00:00',
 'name': 'Salt Lake City Grilled Cheese',
 'description': 'A classic grilled cheese sandwich with a Salt Lake City twist!',
 'ingredients': '2 slices of sourdough bread, 2 slices of cheddar cheese, 2 slices of bacon, 1 tablespoon of butter, 1 teaspoon of garlic powder',
 'directions': '1. Preheat a skillet over medium heat., 2. Spread butter on one side of each slice of bread., 3. Place one slice of bread, butter side down, in the skillet., 4. Top with one slice of cheese, bacon, and the other slice of cheese., 5. Sprinkle garlic powder over the top of the cheese., 6. Place the other slice of bread, butter side up, on top of the cheese., 7. Cook for 3-4 minutes, or until the bottom slice of bread is golden brown., 8. Flip the sandwich and cook for an additional 3-4 minutes, or until the other side is golden brown., 9. Serve and enjoy!',
 'cooking_skill': 'Beginner',
 'place_city': 'Salt Lake City',
 'preparation_time': 'Medi

In [115]:
test['description']

'A classic grilled cheese sandwich with a Salt Lake City twist!'

In [109]:
ingredients_list = test['ingredients'].split(', ')

for ingredient in ingredients_list:
    print(ingredient)

2 slices of sourdough bread
2 slices of cheddar cheese
2 slices of bacon
1 tablespoon of butter
1 teaspoon of garlic powder
